# TODO

In [87]:
import os
import sys
import glob
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.stats import ttest_1samp
from pyscenic.cli.utils import load_signatures
from pyscenic.utils import add_correlation
from tqdm import tqdm
from scipy.stats import ttest_1samp
tqdm.pandas()

In [89]:
def calc_p_value(importances):
    _, p_value = ttest_1samp(importances, 0)
    return p_value

In [79]:
def regulon2sadj(
    regulons,
):
    net_lst = []
    for tf in regulons:
        tf_name = tf.name.split("(")[0]
        tf_targets = tf.gene2weight
        for target, weight in tf_targets.items():
            net_lst.append([tf_name, target, weight])
    net = pd.DataFrame(net_lst, columns=["TF", "target", "importance"])
    return net

In [121]:
example_out_dir = "/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J"
example_ex_mtx_file = "/cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/raw/balanced_genotype_microglia-B6J.0.05.raw.loom"

In [122]:
adata = sc.read_loom(example_ex_mtx_file, sparse=True)

In [123]:
reg_csvs = sorted(glob.glob(os.path.join(example_out_dir, "*.csv")))
reg_csvs

['/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run1_reg.csv',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run2_reg.csv',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run3_reg.csv',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run4_reg.csv',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run5_reg.csv']

In [124]:
all_edges = pd.DataFrame()
for reg_csv in reg_csvs:
    regulons = load_signatures(reg_csv)
    adj_df = regulon2sadj(regulons)
    all_edges = pd.concat([all_edges, adj_df])
all_edges.head()

Create regulons from a dataframe of enriched features.
Additional columns saved: []
Create regulons from a dataframe of enriched features.
Additional columns saved: []
Create regulons from a dataframe of enriched features.
Additional columns saved: []
Create regulons from a dataframe of enriched features.
Additional columns saved: []
Create regulons from a dataframe of enriched features.
Additional columns saved: []


,TF,target,importance
0,Arnt,Setd5,2.911792
1,Arnt,Gm21992,4.975446
2,Arnt,Timm23,2.968481
3,Arnt,Rnf19a,2.271195
4,Arnt,Camsap2,3.600412


In [125]:
# Group by source and target and filter edges that appear in only one run
grouped = all_edges.groupby(['TF', 'target'])
filtered = grouped.filter(lambda x: len(x) > 1)
filtered

,TF,target,importance
0,Arnt,Setd5,2.911792
3,Arnt,Rnf19a,2.271195
6,Arnt,Zfp91,4.214775
7,Arnt,Mga,3.106827
8,Arnt,Bnip2,2.733628
...,...,...,...
30215,Zfx,2510009E07Rik,1.757439
30217,Zfx,Unc13c,3.442252
30218,Zfx,Tshz1,0.416782
30219,Zfx,Cacng2,0.337952


In [126]:
print(f"{len(all_edges) - len(filtered)} edges dropped")

21956 edges dropped


In [127]:
filtered = add_correlation(filtered, adata.to_df())

In [128]:
mean_importance = filtered.groupby(['TF', 'target'])['importance'].mean()
mean_corr = filtered.groupby(['TF', 'target'])['rho'].mean()

In [129]:
p_values_series = filtered.groupby(['TF', 'target'])['importance'].progress_apply(calc_p_value) + TINY
p_values = p_values_series.values

  0%|          | 0/35535 [00:00<?, ?it/s]

  0%|          | 72/35535 [00:00<00:49, 718.86it/s]/tmp/ipykernel_933261/825832589.py:2: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  _, p_value = ttest_1samp(importances, 0)
100%|██████████| 35535/35535 [00:10<00:00, 3533.33it/s]


In [130]:
neg_log_p = -np.log10(p_values)
normalized_importance = (mean_importance - mean_importance.min()) / (mean_importance.max() - mean_importance.min())

In [131]:
consolidated = pd.DataFrame({
    'source': mean_importance.index.get_level_values('TF'),
    'target': mean_importance.index.get_level_values('target'),
    'weight_signed': np.nan,
    'weight_unsigned': mean_importance.values,
    'weight_minmax_normalized': normalized_importance.values,
    'p': p_values,
    '-logp': neg_log_p,
    'correlation': mean_corr.values,
    'description': np.nan
}).reset_index(drop=True)

# Scratch

In [ ]:
def scenic_out_to_co(
    df
):
    """Co"""
    rows = []
    for i, row in df.iterrows():
        tf = row[0]
        target_string = row[8]
        target_list = target_string.rstrip("]").lstrip("[").split("),")
        for target in target_list:
            target = target.replace("(", "").replace("'", "")
            target_split = target.split(",")
            targ = target_split[0].strip()
            weight = float(target_split[1].strip().strip(")"))
            rows.append([tf, targ, weight])
    return_df = pd.DataFrame(rows, columns=["source", "target", "weight"])
    return_df["p"] = np.nan
    return_df["coef_abs"] = return_df["weight"].abs()
    return return_df

In [ ]:
adj_tsvs = sorted(glob.glob(os.path.join(example_out_dir, "*.tsv")))
adj_tsvs

['/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run1_adj.tsv',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run2_adj.tsv',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run3_adj.tsv',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run4_adj.tsv',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run5_adj.tsv']

In [ ]:
looms = sorted(glob.glob(os.path.join(example_out_dir, "*.loom")))
looms

['/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run1_pyscenic_output.loom',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run2_pyscenic_output.loom',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run3_pyscenic_output.loom',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run4_pyscenic_output.loom',
 '/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J/run5_pyscenic_output.loom']